<a href="https://colab.research.google.com/github/samnees/Visualisation/blob/main/Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the important stuff
import pandas as pd
import numpy as np
#Install dash
!pip install dash==1.17.0

In [ ]:
# Download both datasets and save them
url = 'https://github.com/samnees/Visualisation/raw/main/StudentsPerformance.csv'
url2 = 'https://github.com/samnees/Visualisation/blob/main/dataset.xlsx?raw=true'
df = pd.read_csv(url)
df2 = pd.read_excel(url2)

In [ ]:
#Make sure that the colab specific code will be used
running_in_colab = True
# Install ngrok A.K.A firewall breaker
!pip install ngrok
# Penetrate the firewall of google colab and free the port 8050 for dash.
get_ipython().system_raw('./ngrok http 8050 &')
# P.S. Ignore the ERROR; it's nothing

  Using cached https://files.pythonhosted.org/packages/67/50/c64abd519f50537df4726bc2572862d1925e377d4a80a9c06e70db048682/ngrok-0.1.6.tar.gz
  Using cached https://files.pythonhosted.org/packages/03/ac/9a3f332f8d7d27cd8929922b2e225842c1623760364b37fc79dc0f65e77d/ping-0.2.tar.gz
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [ ]:
!pip install jupyter-dash
from jupyter_dash import JupyterDash

In [11]:
#Code in this block is responsible for dashboard

#Important check to know when this is run in colab
try:
    running_in_colab
except NameError:
    running_in_colab = False

#Important to import all libraries for the dashboard
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

#Downloading and saving the dataset
url = 'https://github.com/samnees/Visualisation/raw/main/StudentsPerformance.csv'
df = pd.read_csv(url)

#Preprocessing
df.replace(['female', 'male'], [1,0], inplace=True)
df.replace(['standard', 'free/reduced'], [0, 1], inplace=True)
df.replace(['none', 'completed'], [0, 1], inplace=True)
df.replace(['some college', "associate's degree",
            'high school', 'some high school',
            "bachelor's degree", "master's degree"], [0, 1, 2, 3, 4, 5], inplace=True)
df['average score'] = (df['math score'] + df['reading score'] + df['writing score'])/3

cols_list = []
for col in df.columns:
    if col != 'race/ethnicity':
        cols_list.append({'label': col, 'value': col})

if running_in_colab:
    app = JupyterDash(__name__)
else:
    app = dash.Dash()

colors = {  

    'background': '#111111',

    'text': '#7FDBFF'

}

app.layout = html.Div([

    #First row [top]
    html.Div([

        #First column (choosing attributes) [top left]
        html.Div([
            dcc.Markdown('''
            ### **Choose what you want to plot**
            '''),

            dcc.Dropdown(
                id='dd-menu-x',
                options=cols_list,
                value=df.columns[0],
                multi = True  
            ),


        ],style={'padding-left':'1.5%', 'padding-right':'0.5%',"width":"31.333%",
                 "display":"inline-block","background":"lightgreen",
                 "padding-top":"0%", "padding-bottom":"1%"}),

        #Second column [top center]
        html.Div([

            dcc.Markdown('''
            ### **Column 2**
            '''),

            dcc.Dropdown(
                id='dd-menu-3',
                options=[{'label':'x','value':'x'}],
                value=df.columns[0]
            ),

            dcc.Dropdown(
                id='dd-menu-4',
                options=[{'label':'y','value':'y'}],
                value=df.columns[0]
            )

        ],style={'padding-left':'1%', 'padding-right':'1%',"width":"31.333%",
                 "display":"inline-block","background":"lightgreen",
                 "padding-top":"0%", "padding-bottom":"1%"}),

        #Third column [top right]
        html.Div([

            dcc.Markdown('''
            ### **Column 3**
            '''),

            dcc.Dropdown(
                id='dd-menu-5',
                options=[{'label':'x','value':'x'}],
                value=df.columns[0]
            ),

            dcc.Dropdown(
                id='dd-menu-6',
                options=[{'label':'y','value':'y'}],
                value=df.columns[0]
            )

        ],style={'padding-left':'0.5%', 'padding-right':'1.5%',"width":"31.333%",
                 "display":"inline-block","background":"lightgreen",
                 "padding-top":"0%", "padding-bottom":"1%"})

    ], style={"display":"block","background":"green"}),



    #Second row [bottom]
    html.Div([

        #First column [bottom left]
        html.Div([

            dcc.Markdown('''
            ### **Column 4**
            '''),

            dcc.Dropdown(
                id='dd-menu-7',
                options=[{'label':'x','value':'x'}],
                value=df.columns[0]
            ),

            dcc.Dropdown(
                id='dd-menu-8',
                options=[{'label':'y','value':'y'}],
                value=df.columns[0]
            ),

            #Bonus content
            html.Div([
                dcc.Graph(
                id='histogram1'
            )],style={"background":"lightgreen", "padding-top":"2%",
                      "padding-bottom":"0%"}),

            html.Div([
                dcc.Graph(
                id='histogram2'
            )],style={"background":"lightgreen", "padding-top":"2%",
                      "padding-bottom":"1.5%"})

        ],style={'padding-left':'1.5%', 'padding-right':'0.5%',"width":"31.333%",
                 "display":"inline-block","background":"lightgreen",
                 "padding-top":"0%", "padding-bottom":"1%", 'vertical-align': 'top'}),

        #Second column (plot) [bottom right]
        html.Div([
            dcc.Graph(
                id='main-comparison'
            )
        ],style={'padding-left':'1%', 'padding-right':'1.5%',"width":"64.166%",
                 "display":"inline-block","background":"lightgreen",
                 "padding-top":"0%", "padding-bottom":"1%", 'vertical-align': 'top'})

    ], style={"display":"block", "background":"green"})

])

@app.callback(
    dash.dependencies.Output('main-comparison', 'figure'),
    [dash.dependencies.Input('dd-menu-x', 'value'),
     dash.dependencies.Input('dd-menu-x', df.columns[0]='value')])
def update_output(selection1, selection2):
    print('You have selected "{}", "{}"'.format(selection1,
                                                selection2))
    figure={
        'data': [
            go.Scatter(
                x=df[selection1],
                y=df[selection2],

                mode='markers',

                opacity=0.8,

                marker={

                    'size': 15,

                    'line': {'width': 0.5, 'color': 'white'}

                },

                #name=selection

            ) for i in [1]

        ],

        'layout': go.Layout(

            xaxis={'title': selection1},

            yaxis={'title': selection2},

            margin={'l': 40, 'b': 40, 't': 10, 'r': 10},

            legend={'x': 0, 'y': 1},

            hovermode='closest'

        )
    }

    return figure

@app.callback(
    dash.dependencies.Output('histogram1', 'figure'),
    [dash.dependencies.Input('dd-menu-x', 'value')])
def update_output(selection1):
    figure = px.histogram(df, x=selection1)
    return figure

@app.callback(
    dash.dependencies.Output('histogram2', 'figure'),
    [dash.dependencies.Input('dd-menu-x', 'value')])
def update_output(selection2):
    figure = px.histogram(df, x=selection2)
    return figure


if __name__ == '__main__':
    if running_in_colab:
        app.run_server(mode="external")
    else:
        app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
'dd-menu-x


SyntaxError: ignored